<a href="https://colab.research.google.com/github/MaheshGouru/CS5265-Assignment-1/blob/main/CS5265_Assignment_1_Gouru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background
NOTE: From external sources->summarize and cite for assignment submission
The data set measures the running time of a matrix-matrix product A*B = C, where all matrices have size 2048 x 2048, using a parameterizable SGEMM GPU kernel with 241600 possible parameter combinations. For each tested combination, 4 runs were performed and their results are reported as the 4 last columns. All times are measured in milliseconds*.

There are 14 parameter, the first 10 are ordinal and can only take up to 4 different powers of two values, and the 4 last variables are binary. Out of 1327104 total parameter combinations, only 241600 are feasible (due to various kernel constraints). This data set contains the results for all these feasible combinations.

The experiment was run on a desktop workstation running Ubuntu 16.04 Linux with an Intel Core i5 (3.5GHz), 16GB RAM, and a NVidia Geforce GTX 680 4GB GF580 GTX-1.5GB GPU. We use the 'gemm_fast' kernel from the automatic OpenCL kernel tuning library 'CLTune' ([Web Link]).

* Note: for this kind of data sets it is usually better to work with the logarithm of the running times (see e.g. Falch and Elster, 'Machine learning-based auto-tuning for enhanced performance portability of OpenCL applications', 2015).

# Project Description
Attribute Information:

- Independent variables:
1-2. MWG, NWG: per-matrix 2D tiling at workgroup level: {16, 32, 64, 128} (integer)
3. KWG: inner dimension of 2D tiling at workgroup level: {16, 32} (integer)
4-5. MDIMC, NDIMC: local workgroup size: {8, 16, 32} (integer)
6-7. MDIMA, NDIMB: local memory shape: {8, 16, 32} (integer)
8. KWI: kernel loop unrolling factor: {2, 8} (integer)
9-10. VWM, VWN: per-matrix vector widths for loading and storing: {1, 2, 4, 8} (integer)
11-12. STRM, STRN: enable stride for accessing off-chip memory within a single thread: {0, 1} (categorical)
13-14. SA, SB: per-matrix manual caching of the 2D workgroup tile: {0, 1} (categorical)

- Output:
15-18. Run1, Run2, Run3, Run4: performance times in milliseconds for 4 independent runs using the same parameters. They range between 13.25 and 3397.08.
# Performance Metric(s)
A fraction of this data set was used in the following paper to compute a tensor train based predictive model and estimate the Sobol sensitivity indices of all the parameters:

Rafael Ballester-Ripoll, Enrique G. Paredes, Renato Pajarola.
Sobol Tensor Trains for Global Sensitivity Analysis.
In arXiv Computer Science / Numerical Analysis e-prints, 2017
([Web Link]).

#References
https://archive.ics.uci.edu/ml/datasets/SGEMM+GPU+kernel+performance

https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html

---

## Clarification on Datasets needed for the Project Assignments
Just to recap some important highlights of the datasets needed for the upcoming project assignments:

- Size: at least 5000 datapoints (samples/rows/records), less than 100,000 datapoints are recommended

- #&nbsp;of columns: at least 7 attributes/features/variables (columns) are needed 

- Types of columns: a good dataset will contain not only numerical columns but also categorical columns (e.g., gender, dates, or anything else that has a finite number of categories) 

Some resources for acquiring data: 

- https://www.kaggle.com/datasets

- https://datasetsearch.research.google.com/ 

- https://archive.ics.uci.edu/ml/datasets.php 
---